In [ ]:
import numpy as np
import tensorflow as tf
import sys, re
import os
import glob

The goal is to classify (binary) numbers as even and odd

In [ ]:
train_data = np.asarray([[0,0,0],[0,0,1],[0,1,0],[0,1,1],[1,0,0],[1,0,1],[1,1,0],[1,1,1]], dtype=np.float32)
train_labels= np.asarray([0      ,1      ,0      ,1      ,0      ,1      ,0      ,1], dtype=np.int32)

In [ ]:
import shutil
shutil.rmtree("examplemodel")

Your turn: add a dense layer to the below:

In [ ]:
def model_function(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 1])
    logits = tf.layers.dense(inputs=hidden, units=2)
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
        train_op = optimizer.minimize( loss=loss, global_step=tf.train.get_global_step() )
        return tf.estimator.EstimatorSpec( mode=mode, loss=loss, train_op=train_op )

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [ ]:
estimator = tf.estimator.Estimator(model_fn=model_function, model_dir="examplemodel")
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=10,
    num_epochs=None,
    shuffle=True)


In [ ]:
estimator.train(
  input_fn=train_input_fn,
  steps=10000)

In [ ]:
test_data = [[0,1,1]]
test_data = np.asarray(test_data, dtype=np.float32)
eval_input_fn = tf.estimator.inputs.numpy_input_fn( x={"x":test_data}, num_epochs=1, shuffle=False )
eval_results = estimator.predict(input_fn=eval_input_fn)
result = eval_results.__next__()
print("The number "+str(test_data[0])+" is odd: "+str(result["classes"]))

In [ ]:
print(str(estimator.get_variable_names()))
print("WEIGHTS 0"+str(estimator.get_variable_value("dense/kernel")))
print("BIAS 0"+str(estimator.get_variable_value("dense/bias")))
print("WEIGHTS 1"+str(estimator.get_variable_value("dense_1/kernel")))
print("BIAS 1"+str(estimator.get_variable_value("dense_1/bias")))